# First

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer,pipeline,AdamW , get_scheduler
import torch
from accelerate import Accelerator
from tqdm import tqdm
from datasets import load_dataset

In [3]:
ds = load_dataset("issai/kazqad", "kazqad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

(…)ing-comprehension-v1.0-kk-train.jsonl.gz:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

(…)omprehension-v1.0-kk-validation.jsonl.gz:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ding-comprehension-v1.0-kk-test.jsonl.gz:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3163 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/764 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2713 [00:00<?, ? examples/s]

In [4]:
df_train= ds['train'].to_pandas()
df_test= ds['test'].to_pandas()
df_val= ds['validation'].to_pandas()

In [5]:
repo_id = 'nur-dev/roberta-kaz-large'
model = AutoModelForQuestionAnswering.from_pretrained(repo_id)
tokenizer = AutoTokenizer.from_pretrained(repo_id)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at nur-dev/roberta-kaz-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [6]:
df_train=df_train.drop(['id', 'title'],axis=1)

In [7]:
df_train.head()

,context,question,answers
0,Belcalis Marlenis Almánzar ( /ˈbɛlkəliːz ɑːlˈm...,Карди Би музыканың қай жанрында ән айтады?,"{'text': ['рэп'], 'answer_start': [164]}"
1,Карди Би дискографиясы негізінен трап пен R&am...,Карди Би музыканың қай жанрында ән айтады?,{'text': ['трап пен R&amp;B элементтерін қамти...
2,Тікбұрышты горизонтальды үлендігі бірдей бірін...,Қызыл жолақты американдық жалау нені білдіреді?,"{'text': ['шыдамдылық пен батырлықты'], 'answe..."
3,Мо́на Ли́за (Mona Lisa) — бұл шамамен 1503 жыл...,Әйгілі 'Мона Лиза' картинасы қайда қойылған?,"{'text': ['Франция, Париж)'], 'answer_start': ..."
4,Ливерпуль футбол клубы (Liverpool Football Clu...,Ливерпуль Премьер-Лига титулын қанша рет жеңіп...,"{'text': ['19 рет'], 'answer_start': [208]}"


In [8]:
df_train.head()

,context,question,answers
0,Belcalis Marlenis Almánzar ( /ˈbɛlkəliːz ɑːlˈm...,Карди Би музыканың қай жанрында ән айтады?,"{'text': ['рэп'], 'answer_start': [164]}"
1,Карди Би дискографиясы негізінен трап пен R&am...,Карди Би музыканың қай жанрында ән айтады?,{'text': ['трап пен R&amp;B элементтерін қамти...
2,Тікбұрышты горизонтальды үлендігі бірдей бірін...,Қызыл жолақты американдық жалау нені білдіреді?,"{'text': ['шыдамдылық пен батырлықты'], 'answe..."
3,Мо́на Ли́за (Mona Lisa) — бұл шамамен 1503 жыл...,Әйгілі 'Мона Лиза' картинасы қайда қойылған?,"{'text': ['Франция, Париж)'], 'answer_start': ..."
4,Ливерпуль футбол клубы (Liverpool Football Clu...,Ливерпуль Премьер-Лига титулын қанша рет жеңіп...,"{'text': ['19 рет'], 'answer_start': [208]}"


In [11]:
df_train = pd.concat([df_train, pd.json_normalize(df_train['answers'])], axis=1).drop(columns=['answers'])
df_train['text'] = df_train['text'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_train['answer_start'] = df_train['answer_start'].apply(lambda x: x[0] if isinstance(x, list) else x)
for column in ['text','answer_start']:
    df_train[column]= df_train[column].apply(lambda x : x[0])

In [12]:
df_test = pd.concat([df_test, pd.json_normalize(df_test['answers'])], axis=1).drop(columns=['answers'])
df_test['text'] = df_test['text'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_test['answer_start'] = df_test['answer_start'].apply(lambda x: x[0] if isinstance(x, list) else x)
for column in ['text','answer_start']:
    df_test[column]= df_test[column].apply(lambda x : x[0])

In [13]:
from torch.utils.data import Dataset
class QADataset(Dataset):
    def __init__(self, contexts, questions, answers, tokenizer, max_len):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        context = self.contexts[idx]
        question = self.questions[idx]
        answer = self.answers[idx]

        # Tokenize the question-context pair
        encoding = self.tokenizer(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Find the answer's start and end positions in the context
        answer_start_index = context.find(answer)
        answer_end_index = answer_start_index + len(answer)

        # Set start and end positions for the answer tokens
        start_positions = encoding.char_to_token(0, answer_start_index) if answer_start_index != -1 else 0
        end_positions = encoding.char_to_token(0, answer_end_index - 1) if answer_end_index != -1 else 0

        # If start or end positions are not found, set them to 0
        if start_positions is None:
            start_positions = 0
        if end_positions is None:
            end_positions = 0

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'start_positions': torch.tensor(start_positions, dtype=torch.long),
            'end_positions': torch.tensor(end_positions, dtype=torch.long),
        }


In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Split the data

# Create datasets
train_dataset = QADataset(
    contexts=df_train['context'].tolist(),
    questions=df_train['question'].tolist(),
    answers=df_train['text'].tolist(),
    tokenizer=tokenizer,
    max_len=128
)

test_dataset = QADataset(
    contexts=df_test['context'].tolist(),
    questions=df_test['question'].tolist(),
    answers=df_test['text'].tolist(),
    tokenizer=tokenizer,
    max_len=128
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [15]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch

# Set the device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # Number of training epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

for epoch in range(3):  # Number of epochs
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}")


  0%|          | 0/198 [00:00<?, ?it/s]

In [ ]:
def compute_exact_match(prediction, ground_truth):
    return int(prediction.strip() == ground_truth.strip())

def compute_f1(prediction, ground_truth):
    pred_tokens = prediction.split()
    gt_tokens = ground_truth.split()
    common_tokens = set(pred_tokens) & set(gt_tokens)

    if len(common_tokens) == 0:
        return 0

    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(gt_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

model.eval()
total_f1 = 0
total_em = 0
accurate_predictions = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        # Get model predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Get the highest-scoring start and end tokens
        predicted_start = torch.argmax(start_logits, dim=1)
        predicted_end = torch.argmax(end_logits, dim=1)

        # Calculate accuracy for start and end positions
        start_accuracy = (predicted_start == start_positions).sum().item()
        end_accuracy = (predicted_end == end_positions).sum().item()
        accurate_predictions += start_accuracy + end_accuracy

        # Convert token positions back to text for EM and F1
        for i in range(input_ids.size(0)):
            input_id = input_ids[i]
            context_text = tokenizer.decode(input_id, skip_special_tokens=True)
            true_answer = tokenizer.decode(input_id[start_positions[i]:end_positions[i]+1], skip_special_tokens=True)
            predicted_answer = tokenizer.decode(input_id[predicted_start[i]:predicted_end[i]+1], skip_special_tokens=True)

            # Compute EM and F1 for each answer
            em = compute_exact_match(predicted_answer, true_answer)
            f1 = compute_f1(predicted_answer, true_answer)
            total_em += em
            total_f1 += f1

            total_samples += 1

# Calculate average scores
accuracy = accurate_predictions / (2 * total_samples)  # Start and end positions both count towards accuracy
exact_match = total_em / total_samples
f1_score_avg = total_f1 / total_samples

print(f"Accuracy: {accuracy:.4f}")
print(f"Exact Match (EM): {exact_match:.4f}")
print(f"F1 Score: {f1_score_avg:.4f}")